In [23]:
import os
from pathlib import Path
from typing import Tuple

import numpy as np
import numpy.typing as npt
import openai
import pandas as pd
from numpy.typing import NDArray
from scipy import sparse

import amazon_dataset

DATASET = 'Clothing_Shoes_and_Jewelry'
DMRL_RES_PATH = Path(f'data/{DATASET}/best_scores_vitbert_dmrl.npz')
SEMMACRID_RES_PATH = Path(f'data/SEM-MacridVAE-{DATASET}-run/eval.npz')
TRAIN_PATH = Path(f'data/{DATASET}/train.txt')
TEST_PATH = Path(f'data/{DATASET}/test.txt')
ITEMS_PATH = Path(f'data/{DATASET}/items.txt')
USERS_PATH = Path(f'data/{DATASET}/users.txt')

In [8]:
assert DMRL_RES_PATH.exists()
assert SEMMACRID_RES_PATH.exists()
assert TRAIN_PATH.exists()
assert TEST_PATH.exists()
assert ITEMS_PATH.exists()
assert USERS_PATH.exists()
assert 'OPENAI_APIKEY' in os.environ

#  Load all data

In [35]:
dmrl_results: NDArray[np.floating] = np.vstack(next(iter(np.load(DMRL_RES_PATH, allow_pickle=True).values())))
dmrl_results.shape

(23318, 38493)

In [4]:
sem_macrid_results: NDArray[np.floating] = next(iter(np.load(SEMMACRID_RES_PATH, allow_pickle=True).values()))
sem_macrid_results.shape

(23318, 38493)

In [5]:
assert sem_macrid_results.shape == dmrl_results.shape

Get users

In [14]:
users = pd.read_csv(USERS_PATH)
#assert len(users) == sem_macrid_results.shape[0]
users

,Unnamed: 0,0
0,A30NKRF3KBGA06,0
1,AL0XGCBE6Z22M,1
2,AMT5LF0TKY67C,2
3,A2BY8EVXA3NRHD,3
4,AWE6KR1ELIYQ3,4
...,...,...
23313,A1MFBF49ZFMH2N,23313
23314,A36AF5I7D0VO8F,23314
23315,A35ZS7JT3G9B8,23315
23316,A3GC94SEKQI3QU,23316


Get items

In [15]:
items = pd.read_csv(ITEMS_PATH, index_col=0)
#assert len(items) == sem_macrid_results.shape[1]
items

,item_id
B000B6AV7K,0
B0143D7EE4,1
B0105V2DEY,2
B014EY21H2,3
B005LUROIK,4
...,...
B00A9R2P7A,38488
B017HK485S,38489
B008H7UKYY,38490
B006K6PJTK,38491


In [16]:
def load_interaction_matrix(file: Path, shape: Tuple[int, int]) -> sparse.csr_matrix:
    df = pd.read_csv(file)
    return sparse.csr_matrix(
        (np.ones_like(df['user']), (df['user'], df['item'])),
        shape=shape,
        dtype=float
    )


train = load_interaction_matrix(TRAIN_PATH, (len(users), len(items)))
test = load_interaction_matrix(TEST_PATH, (len(users), len(items)))

# Analyze results

Let's make the trained data to be zero 

In [71]:
dmrl_results[train.nonzero()] = -np.inf
#sem_macrid_results[train.nonzero()] = -np.inf

Analyze how they agree in the top 5 results

In [10]:
k = 5

best_dmrl = dmrl_results.argpartition(kth=-k, axis=1)[:, -k:]
#best_sem_macridvae = sem_macrid_results.argpartition(kth=-k, axis=1)[:, -k:]

# Get metrics

In [348]:
def get_top_k_items(x: npt.NDArray, k: int) -> npt.NDArray:
    # Best indexes without sorting
    best_indices = np.argpartition(x, axis=1, kth=-k)[:, -k:]

    # Best scores sorted in ascending order
    best_values = np.take_along_axis(x, best_indices, axis=1)

    # Best indices in descending order (from best_best_values)
    best_values_idxs = np.argsort(best_values, axis=1)[:,-1:-k-1:-1]

    # Now, we have the best indices in descending order
    return np.take_along_axis(best_indices, best_values_idxs, axis=1)

def ndcg_at_k(scores: npt.NDArray, test: sparse.csr_matrix, k: int) -> float:
    assert scores.shape == test.shape
    best_scores = get_top_k_items(scores, k)
    
    test = test > 0
    num_positives = np.minimum(k, test.sum(axis=1)).astype(np.int32)
    
    rows = np.indices(best_scores.shape)[0]
    denominator = np.log2(np.arange(2, k + 2))
    
    idcg = np.cumsum(1 / denominator)[np.maximum(num_positives - 1, 0)]
    
    dcg = np.sum(test[rows, best_scores] / denominator, axis=1)

    ndcg = dcg / idcg
    return ndcg

def recall_at_k(scores: npt.NDArray, test: sparse.csr_matrix, k: int) -> float:
    assert scores.shape == test.shape
    best_scores = get_top_k_items(scores, k)
    
    test = test > 0
    num_positives = np.minimum(k, test.sum(axis=1)).astype(np.int32)
    
    rows = np.indices(best_scores.shape)[0]
    recall = np.sum(test[rows, best_scores], axis=1) / num_positives
    return recall

def precision_at_k(scores: npt.NDArray, test: sparse.csr_matrix, k: int) -> float:
    assert scores.shape == test.shape
    best_scores = get_top_k_items(scores, k)
    
    test = test > 0
    
    rows = np.indices(best_scores.shape)[0]
    precision = np.sum(test[rows, best_scores], axis=1) / k
    return precision

def average_precision(scores: npt.NDArray, test: sparse.csr_matrix) -> float:
    assert scores.shape == test.shape
    
    test = test > 0
    num_positives = np.asarray(test.sum(axis=1).astype(np.int32)).reshape(-1)
    
    k = num_positives.max()
    best_scores = get_top_k_items(scores, k)
    
    rows = np.indices(best_scores.shape)[0]
    labels = test[rows, best_scores].toarray()
    
    true_positives_at_k = np.cumsum(labels, axis=1)
    precisions_at_k  = true_positives_at_k / np.arange(1, k+1)
    return (precisions_at_k * labels).sum(axis=1) / num_positives


print(f'NDCG@5:      {ndcg_at_k(dmrl_results, test, 5).mean():.6f}')
print(f'Recall@5:    {recall_at_k(dmrl_results, test, 5).mean():.6f}')
print(f'MAP:         {average_precision(dmrl_results, test).mean():.6f}')


NDCG@5:      0.203675
Recall@5:    0.240542
MAP:         0.190090


In [362]:
def build_metrics(results_path: Path):
    print(f'Loading results from {results_path}')
    results = np.vstack(next(iter(np.load(results_path, allow_pickle=True).values())))
    print(f'Computing metrics for {results_path}')
    return {
            'path': results_path.stem,
            'ndcg@5': ndcg_at_k(results, test, 5).mean(),
            'recall@5': recall_at_k(results, test, 5).mean(),
            'map': average_precision(results, test).mean(),
        }

def build_results_matrix():
    metrics = [
        build_metrics(p) 
        for p in Path(f'data/{DATASET}').glob('best_scores_*.npz')
    ]
    return pd.DataFrame.from_records(metrics, index='path')

all_metrics = build_results_matrix()
all_metrics

Loading results from data/Clothing_Shoes_and_Jewelry/best_scores_vitbert_dmrl.npz
Computing metrics for data/Clothing_Shoes_and_Jewelry/best_scores_vitbert_dmrl.npz
Loading results from data/Clothing_Shoes_and_Jewelry/best_scores_clipimage.npz
Computing metrics for data/Clothing_Shoes_and_Jewelry/best_scores_clipimage.npz
Loading results from data/Clothing_Shoes_and_Jewelry/best_scores_nomodality.npz
Computing metrics for data/Clothing_Shoes_and_Jewelry/best_scores_nomodality.npz
Loading results from data/Clothing_Shoes_and_Jewelry/best_scores_clip.npz
Computing metrics for data/Clothing_Shoes_and_Jewelry/best_scores_clip.npz
Loading results from data/Clothing_Shoes_and_Jewelry/best_scores_vitbert.npz
Computing metrics for data/Clothing_Shoes_and_Jewelry/best_scores_vitbert.npz
Loading results from data/Clothing_Shoes_and_Jewelry/best_scores_cliptext_dmrl.npz
Computing metrics for data/Clothing_Shoes_and_Jewelry/best_scores_cliptext_dmrl.npz


,ndcg@5,recall@5,map
path,,,
best_scores_vitbert_dmrl,0.143835,0.224602,0.126067
best_scores_clipimage,0.148636,0.228908,0.129358
best_scores_nomodality,0.148186,0.231940,0.127806
best_scores_clip,0.105895,0.165693,0.094644
best_scores_vitbert,0.143909,0.228288,0.125099
best_scores_cliptext_dmrl,0.147130,0.226622,0.129282


Arrays are very memory intensive so we delete the original results

In [14]:
import gc
del sem_macrid_results, dmrl_results
gc.collect()

0

In [16]:
best_dmrl

array([[11253, 19594, 15314, 29368, 12176],
       [32501, 34301,  2121, 23339,  8913],
       [13181, 30484,  3235, 32930, 36164],
       ...,
       [ 3482,  6105,  4866, 14556, 24574],
       [32060,  5807, 24346, 13147, 32438],
       [ 6217, 33684, 27597, 13116, 34301]])

In [17]:
best_sem_macridvae

array([[ 8703, 10284, 26373, 38308,  1752],
       [  854, 30552, 26624, 32501, 34301],
       [ 8703, 10284,  1752, 38308, 26373],
       ...,
       [ 8703, 26373, 10284, 38308,  1752],
       [ 8703, 10284, 26373,  1752, 38308],
       [16534, 13116, 34301, 33684, 27597]])

In [46]:
best_counts_semmacridvae = pd.Series(best_sem_macridvae.flatten()).value_counts(normalize=True).to_frame('percentage').head(20)
best_counts_semmacridvae

,percentage
1752,0.086302
38308,0.083978
26373,0.070135
10284,0.066121
8703,0.064560
34301,0.029874
32501,0.027481
30552,0.025868
854,0.021666
26624,0.021271


In [47]:
best_counts_dmrl = pd.Series(best_dmrl.flatten()).value_counts(normalize=True).to_frame('percentage').head(20)
best_counts_dmrl

,percentage
6027,0.006896
21557,0.005412
17254,0.005344
921,0.005078
6112,0.005035
24346,0.004992
24956,0.004975
31411,0.004975
8963,0.004666
18492,0.004520


In [48]:
set(best_counts_dmrl.index) & set(best_counts_semmacridvae.index)

{6027, 6112, 17254, 38308}

In [32]:
pd.Series(best_dmrl.flatten()).value_counts() > 10

array([21799,     0,     2, ..., 38484,     3, 38482])

In [91]:
pd.Series(np.asarray(test.sum(axis=0))[0].astype(int)).sort_values(ascending=False).head(20)

18031    168
38308    165
26373    164
3520     163
8703     160
34043    153
28568    153
10284    152
6388     152
5372     146
1752     140
12593    140
13038    138
23510    131
33683    130
307      123
19838    118
35166     91
31411     82
6536      78
dtype: int64

In [21]:
pd.Series(train.sum(axis=0).tolist()[0]).sort_values(ascending=False)

12593    617.0
33683    598.0
19838    590.0
307      569.0
10284    567.0
         ...  
31833      0.0
8552       0.0
8557       0.0
8558       0.0
21618      0.0
Length: 38493, dtype: float64

In [72]:
items.iloc[2]

Unnamed: 0    B0105V2DEY
item_id                2
Name: 2, dtype: object

In [12]:
products = amazon_dataset.products_df(DATASET)
products

,asin,description,title,brand,main_cat,rank,price,image_slug,image_url,feature,category,tech_detail
id,,,,,,,,,,,,
47,5120053084,Top Length (Neck to Bottom Hem) Small - 29 inc...,sofsy Soft-Touch Rayon Blend Tie Front Nursing...,None,None,"87,615inClothing,ShoesJewelry(",None,"[51HJbA8UG2L, 51FufN7RbSL, 51vKjwQ6eAL, 410fEp...",[https://images-na.ssl-images-amazon.com/image...,[SAVE 10% WHEN YOU ADD 2 OR MORE sofsy PRODUCT...,"[Clothing, Shoes & Jewelry, Women, Maternity, ...",None
144,7709260373,None,LJYH Children's Collar Motorcycle Faux Leather...,LJYH,None,"15,484inClothing,ShoesJewelry(",$23.99 - $29.99,"[41MAnSZ8QTL, 51qL9W098pL, 51QWVSqlAML, 51eOan...",[https://images-na.ssl-images-amazon.com/image...,"[100% New Faux Leather Coat, Fashion New Slim ...","[Clothing, Shoes & Jewelry, Boys, Clothing, Ja...",None
284,B00001TOXD,Includes One Broom. This broom goes great with...,Adult Witch Broom,None,Toys & Games,None,$4.99,[21VAYWoNedL],[https://m.media-amazon.com/images/I/21VAYWoNe...,[Brand new authentic licensed Pegan Witch broo...,"[Clothing, Shoes & Jewelry, Costumes & Accesso...",None
316,B00004U1J2,Tired of having your kid running around the ho...,Buzz Lightyear Jet Pack,None,Toys & Games,None,$7.41,[31y%2BZwVQJJL],[https://images-na.ssl-images-amazon.com/image...,"[Polyester, Imported, 16"" high, 12"" wide, Qual...","[Clothing, Shoes & Jewelry, Costumes & Accesso...",None
333,B00004VWJ3,Birkenstock's Black Super Birki Clog is made o...,"Birki's Super Pu, Black, 44 M EU (13 Women /11...",Birki's,None,">#3,164 in Patio, Lawn & Garden (See Top 100 i...",None,"[51g%2BtUP7YSL, 41Jn0zjKRuL, 41ElBt2gWdL, 31Aw...",[https://images-na.ssl-images-amazon.com/image...,[Birkenstock SUPER BIRKI BLACK PU 44],"[Clothing, Shoes & Jewelry, Women, Shoes, Mule...",None
...,...,...,...,...,...,...,...,...,...,...,...,...
2684954,B01HJDBMUM,When you need a fashionable shoe for your son-...,Deer Stags Kid's Brilliant Dress Comfort Oxfor...,None,None,"343,935inClothing,ShoesJewelry(",$16.80 - $40.00,"[51Q9Np4I4OL, 41JgD3Klb6L, 31hD-OLssoL, 41ABLP...",[https://images-na.ssl-images-amazon.com/image...,"[100% Manmade, Synthetic sole, QUALITY YOU CAN...","[Clothing, Shoes & Jewelry, Girls, Shoes, Oxfo...",None
2684955,B01HJCZ02O,"Dynamic in design, the honor sneaker is comfor...",Dansko Women's Honor Sneaker,None,None,"44,554inClothing,ShoesJewelry(",$74.93 - $221.43,"[51wGv%2BSmTHL, 41AE5SULqtL, 311xxeLqZ2L, 41Js...",[https://images-na.ssl-images-amazon.com/image...,"[100% Leather, Imported, Rubber sole, Shaft me...","[Clothing, Shoes & Jewelry, Women, Shoes, Fash...",None
2684979,B01HJDZM30,Part of our Performance Fishing Gear line of p...,Columbia PFG Mesh Snap Back Ball Cap,Columbia,Sports & Outdoors,"5,346inSportsOutdoors(",$20.90,"[51cajtyCrBL, 416yFSOGLdL, 41t%2Bt3iFPYL, 513t...",[https://images-na.ssl-images-amazon.com/image...,"[blend, ADVANCED TECHNOLOGY: Columbia's PFG Me...","[Clothing, Shoes & Jewelry, Men, Accessories, ...",None


# Generate recommendation explanations

In [15]:
some_recommendations_itemids =  854, 30552, 26624, 32501, 34301
some_asins = items.loc[items['item_id'].isin(some_recommendations_itemids)].index
some_asins

Index(['B013G9AYPC', 'B013G9BBA4', 'B013CA13K0', 'B013CZSN5S', 'B0113OB89S'], dtype='object')

In [18]:
reviews = amazon_dataset.reviews_df(DATASET)
reviews

,id,asin,reviewerID,reviewerName,overall,text,reviewTime,summary,verified,vote
0,676,5120053084,A35EUS1E3WK1HC,Kiley and Mars,5.0,"It's a cute top, works good for nursing and la...",2018-04-10,Decent lounge around top,True,NaN
1,677,5120053084,AKIZYAIS4SYVF,Bethany,5.0,Looks really cute and super easy to nurse my d...,2018-03-22,Cute,True,NaN
2,679,5120053084,A2L74OWEP7H1VC,Shelby0516,3.0,The tie is longer than the pictures showed. Ha...,2018-03-14,Awkward tie,True,NaN
3,681,5120053084,A260RMKZXGDHVH,Kelly Kennedy,5.0,Of all the nursing shirts I bought my daughter...,2018-03-07,this one is her favorite. She wears it with pa...,True,NaN
4,1300,7709260373,A13QI8GT2FFGN6,Amy,5.0,For the price... this is awesome!,2018-03-13,this is awesome!,True,NaN
...,...,...,...,...,...,...,...,...,...,...
178939,32291840,B01HJCSCLK,AAHWQ4FMWLNH3,amazonlover,5.0,"Beautiful. Strong, durable, and chic but subtl...",2018-07-25,Beautiful,True,NaN
178940,32291855,B01HJDVCJI,A2WUHKA1I75SL3,FRCP,3.0,Fit is great on these and they are very comfor...,2018-09-03,Comfortable,False,NaN
178941,32291863,B01HJDVCJI,A7B48AJT6IC0A,Lives2read,4.0,Excellent arch support. Unique tongue design c...,2018-08-13,Unique look and comfort,False,NaN
178942,32291875,B01HJDZM30,A2CJOG4NUHVDGK,Brittney Mitchell,5.0,Bought this for my husband and he absolutely l...,2018-08-29,Five Stars,False,NaN


In [16]:
products.loc[products['asin'].isin(some_asins)]['title']

id
2024956               TOMS Men's Classic Canvas Slip-On
2077409    TOMS Seasonal Classics Women's Slip on Shoes
2078298               TOMS Men's Classic Canvas Slip-On
2081419    TOMS Seasonal Classics Women's Slip on Shoes
2081430    TOMS Seasonal Classics Women's Slip on Shoes
Name: title, dtype: object

In [23]:
def some_reviews_joined(reviews_df: pd.DataFrame, asins: pd.Series):
    def is_long_enough(text: str):
        if text is None:
            return False
        return len(text.split()) > 5

    condition = (
        reviews_df['asin'].isin(asins) & 
        (reviews_df['overall'] >= 4) & 
        (~reviews_df['text'].isna()) &
        reviews_df['text'].apply(is_long_enough)
    )
    some_reviews = reviews.loc[condition]
    some_review_texts = some_reviews['text'].sample(n=5).apply(lambda x: f'- {x}')
    return "\n".join(some_review_texts.to_list())

def prompt_for_completion(reviews_df: pd.DataFrame, asins: pd.Series):
    reviews_joined = some_reviews_joined(reviews_df, asins)
    prefix = "The item has the following comments about it"
    instruction = 'The suggestion for this item is:'
    prompt = f"{prefix}:\n\n{reviews_joined}\n\n{instruction}\n\n"
    print(prompt)
    openai.api_key = os.getenv("OPENAI_APIKEY")
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=prompt,
        temperature=0,
        max_tokens=60,
        top_p=1,
        frequency_penalty=0.5,
        presence_penalty=0
    )
    choices = response["choices"]
    assert len(choices) == 1
    return choices[0]["text"]

print(prompt_for_completion(reviews, some_asins))

The item has the following comments about it:

- Comfortable and adaptable to skirts, shorts, dresses.
- Comfortable and light. My daughter loves them!
- I just received these and I'm in the process of breaking them in.  They are a TAD small with socks on.  I normally wear a size 11, and my toe is basically to the end.  I love the comfort otherwise.  They are very light and are a good hybrid slipper/shoe.
- Great shoe for weekends/lazy days...everyday.  Shoe fitted closely a first but with continue wear it now fit perfectly.  The only thing that disappointed me was the colour faded after its first clean.
- Must have summer shoe!! Perfect Toms style and comfort!!

The suggestion for this item is:


This item is perfect for those looking for a comfortable and adaptable shoe that can be worn with skirts, shorts, and dresses. It is lightweight and provides great comfort. The shoe may fit a bit small at first, but with continued wear it will fit perfectly. The colour may fade after its firs